### **Imported Libraries**

In [2]:
import pandas as pd
import numpy as np

### **Cleaning Worktools**

This section handles all the preparation methods for dealing with missing values and any unreliable data.

In [3]:
def drop_columns(dataframe, columns_to_drop):
    """
    Drop one or multiple columns from a DataFrame.
    
    Parameters:
        dataframe: The input DataFrame.
        columns_to_drop (str or list of str): The column name(s) to drop.
        
    Returns:
        pd.DataFrame: The DataFrame with the specified columns dropped.
    """
    if isinstance(columns_to_drop, str):
        columns_to_drop = [columns_to_drop]

    return dataframe.drop(columns=columns_to_drop, axis=1)


This method works on dropping the whole column(s) from the dataframe. It's going to be used once we encounter one or multiple fields that are fully null or most of the rows are null.

In [4]:
def drop_nan_rows(dataframe, subset):
    """
    Drop rows that contain NaN/NaT or all blank values (spaces) in the DataFrame.
    
    Parameters:
        dataframe: The input DataFrame.
        subset (list of str): The column names to consider when dropping rows.
        
    Returns:
        pd.DataFrame: The DataFrame with rows containing NaN or NaT values dropped.
    """
    # Drop rows with any NaN or NaT values
    dataframe = dataframe.dropna(subset=subset)

    # Drop rows where all values are blank spaces
    condition = dataframe.apply(lambda row: row.str.strip().eq('')).all(axis=1)
    dataframe = dataframe[~condition]

    return dataframe

This method serves to removing just a number of rows within a subset of fields that contains null values (NaN, NaT) or blank values.

In [5]:
def fill_missing_values(dataframe):
    """
    Replace missing values in a DataFrame with appropriate fill values based on data type.
    
    For numerical columns, the missing values will be filled with the median.
    For categorical columns, the missing values will be filled with the most frequent value.
    
    Parameters:
        dataframe: The input DataFrame.
        
    Returns:
        The DataFrame with missing values replaced.
    """
    # Replace blank spaces with NaN
    dataframe = dataframe.replace('', np.nan)
    
    # Fill missing values in numerical columns with the median
    numeric_columns = dataframe.select_dtypes(include=[np.number])
    dataframe[numeric_columns.columns] = dataframe[numeric_columns.columns].fillna(numeric_columns.median())
    
    # Fill missing values in categorical columns with the most frequent value
    categorical_columns = dataframe.select_dtypes(include=['object'])
    dataframe[categorical_columns.columns] = dataframe[categorical_columns.columns].fillna(categorical_columns.mode().iloc[0])
    
    return dataframe


This method is being defined to replace missing values in a DataFrame with appropriate fill values based on data type. As a choice, we are tending to replace the numerical columns with the median and the categorical columns with the most frequent values (the mode, statistically speaking)


In [6]:
def switch_if_negative_diff(dataframe, dt1, dt2):
    """
    Switches the values of datetime1 (dt1) and datetime2 (dt2) in the dataframe
    if datetime1 is greater than datetime2.

    Args:
        dataframe: the dataframe containing 'datetime1' and 'datetime2' columns.
        dt1 (str): the field's name representing the first datetime
        dt2 (str): the field's name representing the second datetime

    Returns:
        The permuted dataframe with switched datetime values if applicable.
    """
    datetime1 = dataframe[dt1]
    datetime2 = dataframe[dt2]
    if datetime1 > datetime2:
        dataframe[dt1], dataframe[dt2] = datetime2, datetime1
    return dataframe

The role of this method is toggling between 2 datetimes that have negative differential.

In [7]:
def interpolate_datetime_nulls(dataframe, column_name):
    """
    Replace null datetime values in a DataFrame column using time-based interpolation.

    Parameters:
        dataframe: The DataFrame containing the column with null datetime values.
        column_name (str): The name of the column with datetime values to interpolate.

    Returns:
        The DataFrame with null datetime values replaced by interpolated values.
    """
    # Convert the specified column to datetime dtype
    dataframe[column_name] = pd.to_datetime(dataframe[column_name])

    # Perform interpolation for missing values using 'bfill' method
    dataframe[column_name] = dataframe[column_name].fillna(method='bfill')
    
    return dataframe

This method takes a DataFrame and the name of the column with datetime values that need interpolation. Interpolation is a mathematical method that estimates the missing values based on the known values surrounding them.

Pandas provides the `interpolate()` method, which can be used to perform linear or time-based interpolation on datetime columns.

The resulting DataFrame will have the null datetime values replaced with interpolated values based on the surrounding known values and the time differences between them.

In [8]:
def fetch_nan_columns(dataframe, threshold = 0.5):
    """
    Fetches the null columns and Separates the most nullified columns for the others.

    Parameters:
        dataframe: The DataFrame containing the column with null values.
        threshold (float): A metric with which we separate the dataframe 
                           based upon holding missing values.

    Returns:
        Two lists of columns names.
    """
    # Store the number of null values within a dataframe df
    df = pd.DataFrame(dataframe.isna().sum(), columns=['Nb_null'])

    # Separate the number of nuls that surpass the length of the dataframe
    cols_to_drop = df[df['Nb_null'] > detb_data.shape[0] * threshold]

    # Separate the number of nuls that are less than the length of the dataframe except 0
    cols_to_manipulate_nan = df[(df['Nb_null'] <= detb_data.shape[0] * threshold) & (df['Nb_null'] > 0)]

    # Return just the name of columns
    return cols_to_drop.index, cols_to_manipulate_nan.index

This method stands out as a way of retrieving the columns that have missing values

In [9]:
def fill_missing_values_subset(dataframe, subset_columns):
    """
    Replace missing values in a subset of columns of a DataFrame with appropriate fill values based on data type.

    For numerical columns, the missing values will be filled with the median.
    For categorical columns, the missing values will be filled with the most frequent value.

    Args:
        dataframe: The input DataFrame.
        subset_columns: List of column names to fill missing values in.

    Returns:
        The DataFrame with missing values replaced for the specified subset of columns.
    """
    # Fill missing values in numerical columns of the subset with the median
    numeric_columns = dataframe[subset_columns].select_dtypes(include=[np.number])
    dataframe[numeric_columns.columns] = dataframe[numeric_columns.columns].fillna(numeric_columns.median())

    # Fill missing values in categorical columns of the subset with the most frequent value
    categorical_columns = dataframe[subset_columns].select_dtypes(include=['object'])
    dataframe[categorical_columns.columns] = dataframe[categorical_columns.columns].fillna(categorical_columns.mode().iloc[0])

    return dataframe

This method replaces missing values in a subset of columns of a DataFrame with appropriate fill values based on data type.

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

def replace_nan_with_knn_classifier(dataframe, target_column, n_neighbors=5):
    """
    Replace NaN values in the target column of the DataFrame using k-Nearest Neighbors (k-NN) classification for prediction.

    Parameters:
        dataframe (pd.DataFrame): The input DataFrame.
        target_column (str): The name of the column containing NaN values to be predicted. Default is 'LOGIN_USER'.
        n_neighbors (int): The number of neighbors to consider for k-NN classification. Default is 5.

    Returns:
        pd.DataFrame: The DataFrame with NaN values in the specified column replaced with predicted values.
    """
    # Make a copy of the original DataFrame
    df_copy = dataframe.copy()

    # Store the data types of datetime columns for conversion back after label encoding
    datetime_columns = df_copy.select_dtypes(include='datetime64').columns

    # Drop datetime columns temporarily
    df_copy.drop(columns=datetime_columns, inplace=True)

    # Label encode all the categorical columns
    label_encoder = LabelEncoder()
    for col in df_copy.select_dtypes(include='object'):
        df_copy[col] = label_encoder.fit_transform(df_copy[col].astype(str))

    # Separate the DataFrame into null and non-null values in the target column
    df_not_null = df_copy.dropna(subset=[target_column])
    df_null = df_copy[df_copy[target_column].isnull()]

    # Check if there are non-null values in the target column
    if len(df_not_null) == 0:
        raise ValueError("No non-null values found in the target column.")

    # Features and target for non-null values
    features_not_null = df_not_null.drop(target_column, axis=1)
    target_not_null = df_not_null[target_column]

    # Fit the non-null values to the k-NN classifier
    knn_classifier = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn_classifier.fit(features_not_null, target_not_null)

    # Reorder columns in df_null to match the order of features_not_null
    df_null_reordered = df_null[features_not_null.columns]
    print(df_null.head())
    print(len(df_null) > 0)
    # Check if there are any null values to predict
    if len(df_null) > 0:
        # Predict the null values in the target column
        predicted_labels = knn_classifier.predict(df_null_reordered)

        # Convert the predicted labels back to original categorical values using the original label encoder
        predicted_values = label_encoder.inverse_transform(predicted_labels)
        print(predicted_values)
        # Replace NaN values in the target column with predicted values
        df_copy.loc[df_copy[target_column].isnull(), target_column] = predicted_values

    # Convert the target column back to the original labels
    for col in df_copy.select_dtypes(include='object'):
        df_copy[col] = label_encoder.inverse_transform(df_copy[col])

    # Restore the datetime columns
    for col in datetime_columns:
        df_copy[col] = dataframe[col]

    return df_copy


This method works on replacing NaN values in a specific column of the DataFrame using k-Nearest Neighbors (k-NN) imputation.

In [11]:
def replace_label_encoded_values(df_encoded, df_categorical, target_column):
    """
    Replace label encoded values in the DataFrame 'df_encoded' with the corresponding categorical values from 'df_categorical'.

    Parameters:
        df_encoded (pd.DataFrame): The DataFrame containing label encoded values to be replaced.
        df_categorical (pd.DataFrame): The DataFrame containing the corresponding categorical values.
        target_column (str): The name of the target column to replace the values.

    Returns:
        pd.DataFrame: The DataFrame 'df_encoded' with the label encoded values replaced by the categorical values.
    """
    # Create a mapping dictionary to store label encoded values and their corresponding alphabetical values
    mapping = {}

    # Iterate over the target column in both DataFrames simultaneously
    for encoded_val, categorical_val in zip(df_encoded[target_column], df_categorical[target_column]):
        if not pd.isna(encoded_val) and not pd.isna(categorical_val):
            # Store the mapping of label encoded value to the corresponding alphabetical value
            mapping[encoded_val] = categorical_val

    # Replace label encoded values with their corresponding categorical values
    df_encoded[target_column] = df_encoded[target_column].replace(mapping)

    return df_encoded
